In [ ]:
import numpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pytz
import geopandas as gpd
from datetime import datetime, timedelta
import scipy.io as sio
from tqdm import tqdm
import os

from coastsat_keras import *

transect_data = gpd.read_file('data_coastsat/transects_edit.geojson')
transect_data = transect_data[transect_data['id'].str.contains("usa_CA")]
def linestring_to_points(feature,line):
    return line.coords

transect_data['points'] = transect_data.apply(lambda l: linestring_to_points(l['site_id'],l['geometry']), axis=1)
site_id = transect_data['id'].to_numpy()

shore_data_new = pd.read_csv('extended_coastsat_data/All_CA_April_2023.csvx')

In [ ]:
plt.close('all')

fig, ((ax0,ax1,ax2),(ax3,ax4,ax5)) = plt.subplots(nrows=2,ncols=3)
plt.subplots_adjust(wspace=0.3)
plt.subplots_adjust(hspace=0.2)
fig.set_size_inches(11,5)

beach_id = 'usa_CA_0011-0014'
clear_session()
shore_data,tide_heights,wave_data,dist_mop,spec1d_interp,freqz = read_shore_data_new(beach_id,site_id,shore_data_new)
df, df_time = create_df_for_keras(shore_data,tide_heights,wave_data,spec1d_interp,freqz,beach_id)
df_proj,mop_time = create_proj_data(beach_id,site_id)
X_all,X_train,X_val,X_test,y_all,y_train,y_val,y_test,df_mean,df_std,X_proj = create_train_val_test_data(df,df_proj)
model = read_keras_model(beach_id,site_id)

sns.histplot(df_proj['tide'],bins=100,stat='density',ax=ax3,label='Tides')
sns.histplot(tide_heights['tide levels'],bins=100,stat='density',ax=ax3,label='Satellite Data')
ax3.set_xlabel('Tide Level (m)')
ax3.text(-1.4,1.30,'(d)',fontsize=12,color='k')
ax3.text(-1.4,1,'Distribution of\nTides',fontsize=12,color=sns.color_palette("Paired")[1])
ax3.text(-1.4,0.7,'Satellite\nSampling',fontsize=12,color=sns.color_palette("Paired")[7])

ax3.set_xlim([-1.5,1.5])
ax3.set_ylabel('Tidal Sampling\n\nDensity')
# ax3.legend()

shore_data['dates'] = pd.to_datetime(shore_data['dates'])
months_array = shore_data['dates'].dt.month.to_numpy()
bins = np.arange(0.5, 13.5, 1)
month_labels = ['J', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D']
sns.histplot(months_array, bins=bins, kde=False ,ax=ax0)
ax0.set_xticks(ticks=np.arange(1, 13), labels=month_labels)
ax0.text(0.1,65,'(a)',fontsize=12,color='k')
ax0.set_ylabel('Seasonal Sampling\n\nCount')
ax0.set_title('Torrey Pines')
ax0.set_ylim([0,75])

beach_id = 'usa_CA_0176-0165'
clear_session()
shore_data,tide_heights,wave_data,dist_mop,spec1d_interp,freqz = read_shore_data_new(beach_id,site_id,shore_data_new)
df, df_time = create_df_for_keras(shore_data,tide_heights,wave_data,spec1d_interp,freqz,beach_id)
df_proj,mop_time = create_proj_data(beach_id,site_id)
X_all,X_train,X_val,X_test,y_all,y_train,y_val,y_test,df_mean,df_std,X_proj = create_train_val_test_data(df,df_proj)
model = read_keras_model(beach_id,site_id)

sns.histplot(df_proj['tide'],bins=100,stat='density',ax=ax4)
sns.histplot(tide_heights['tide levels'],bins=100,stat='density',ax=ax4)
ax4.text(-1.4,1.2,'(e)',fontsize=12,color='k')
ax4.set_xlabel('Tide Level (m)')
ax4.set_xlim([-1.5,1.5])

shore_data['dates'] = pd.to_datetime(shore_data['dates'])
months_array = shore_data['dates'].dt.month.to_numpy()
bins = np.arange(0.5, 13.5, 1)
month_labels = ['J', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D']
sns.histplot(months_array, bins=bins, kde=False ,ax=ax1)
ax1.set_xticks(ticks=np.arange(1, 13), labels=month_labels)
ax1.text(0.1,110,'(b)',fontsize=12,color='k')
ax1.set_title('Monterey') 

beach_id = 'usa_CA_0279-0027'
# beach_id = 'usa_CA_0176-0165'
clear_session()
shore_data,tide_heights,wave_data,dist_mop,spec1d_interp,freqz = read_shore_data_new(beach_id,site_id,shore_data_new)
df, df_time = create_df_for_keras(shore_data,tide_heights,wave_data,spec1d_interp,freqz,beach_id)
df_proj,mop_time = create_proj_data(beach_id,site_id)
X_all,X_train,X_val,X_test,y_all,y_train,y_val,y_test,df_mean,df_std,X_proj = create_train_val_test_data(df,df_proj)
model = read_keras_model(beach_id,site_id)

sns.histplot(df_proj['tide'],bins=100,stat='density',ax=ax5)
sns.histplot(tide_heights['tide levels'],bins=100,stat='density',ax=ax5)
ax5.text(-1.4,1.1,'(f)',fontsize=12,color='k')
ax5.set_xlabel('Tide Level (m)')
ax5.set_xlim([-1.5,1.5])

shore_data['dates'] = pd.to_datetime(shore_data['dates'])
months_array = shore_data['dates'].dt.month.to_numpy()
bins = np.arange(0.5, 13.5, 1)
month_labels = ['J', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D']
sns.histplot(months_array, bins=bins, kde=False ,ax=ax2)
ax2.set_xticks(ticks=np.arange(1, 13), labels=month_labels)
ax2.text(0.1,70,'(c)',fontsize=12,color='k')
ax2.set_title('King Range')

plt.savefig('figures/seasonal_tidal_sampling.png',bbox_inches='tight')
plt.savefig('figures/seasonal_tidal_sampling.pdf',bbox_inches='tight')

plt.show()